In [1]:
!gcloud auth activate-service-account --key-file=${GOOGLE_APPLICATION_CREDENTIALS}

Activated service account credentials for: [kubeflow-user@kubeflow-demo-256908.iam.gserviceaccount.com]


In [2]:
%%capture pip_install_out

!pip install pip --upgrade
!pip install -r mlpipeline_utils/requirements.txt --upgrade

In [3]:
import logging
import os
import sys
sys.path.append('../mlpipeline_utils')
import uuid

import kfp
import kfp.gcp

from mlpipeline_utils.kfp_components import *


logging.getLogger().setLevel(logging.INFO)

In [4]:
PROJECT_ID = !gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
BASE_GCS_PATH = f'gs://{PROJECT_ID}-dev/'

In [5]:
UTILS_IMAGE = kfp.containers.build_image_from_working_dir(
    working_dir='../mlpipeline_utils',
    base_image=f'gcr.io/{PROJECT_ID}/kfp-base',
)
UTILS_IMAGE

INFO:root:Creating the build context directory: /tmp/tmp77exrzq8
INFO:root:Generating the Dockerfile
INFO:root:Generate build files.
INFO:root:Start a kaniko job for build.
INFO:root:Initialized with in-cluster config.
INFO:root:5 seconds: waiting for job to complete
INFO:root:10 seconds: waiting for job to complete
INFO:root:15 seconds: waiting for job to complete
INFO:root:Kaniko job complete.
INFO:root:Built and pushed image: gcr.io/kubeflow-demo-256908/notebook-kubeflow-rafaelbarreto87-notebook-dev/kfp_container@sha256:b872629b9b6c2767004671cbb8b389b3d8dbb14b1921bef12e8f385b0cee3f88.


'gcr.io/kubeflow-demo-256908/notebook-kubeflow-rafaelbarreto87-notebook-dev/kfp_container@sha256:b872629b9b6c2767004671cbb8b389b3d8dbb14b1921bef12e8f385b0cee3f88'

In [6]:
@kfp.dsl.pipeline()
def papermill_pipeline(
    notebook_path: str,
    output_notebook_path: str,
    papermill_options: str='',
    
):
    papermill_comp(
        notebook_path=notebook_path,
        output_notebook_path=output_notebook_path,
        papermill_base_image=UTILS_IMAGE,
        papermill_options=papermill_options
    ) \
        .apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) \
        .set_cpu_request('2') \
#         .set_gpu_limit(1, 'nvidia')

In [7]:
papermill_demo_gcs_path = os.path.join(BASE_GCS_PATH, 'notebooks', 'papermill_demo.ipynb')
papermill_demo_output_gcs_path = os.path.join(BASE_GCS_PATH, 'notebooks', 'papermill_demo.output.ipynb')
!gsutil cp papermill_demo.ipynb {papermill_demo_gcs_path}

Copying file://papermill_demo.ipynb [Content-Type=application/octet-stream]...
/ [1 files][  1.0 KiB/  1.0 KiB]                                                
Operation completed over 1 objects/1.0 KiB.                                      


In [8]:
run_id = str(uuid.uuid4()).replace('-', '')
kfp.Client().create_run_from_pipeline_func(
    papermill_pipeline,
    arguments={
        'notebook_path': papermill_demo_gcs_path,
        'output_notebook_path': papermill_demo_output_gcs_path,
        'papermill_options': ''
    },
    run_name=f'papermill_demo_{run_id}'
)

<kfp._client.Client.create_run_from_pipeline_package.<locals>.RunPipelineResult at 0x7f29621f1e80>

In [9]:
!gsutil cp {papermill_demo_output_gcs_path} papermill_demo.output.ipynb

Copying gs://kubeflow-demo-256908-dev/notebooks/papermill_demo.output.ipynb...
/ [1 files][  2.2 KiB/  2.2 KiB]                                                
Operation completed over 1 objects/2.2 KiB.                                      
